In [ ]:
!pip install -q tensorflow-text

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
!pip install -q tf-models-official

!pip install -q tensorflow-text

!pip install -q tf-models-official


import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
 df = pd.read_csv("/content/essays.csv", header=None, encoding='cp1252')

In [ ]:
df

,0,1,2,3,4,5,6
0,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
1,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
2,1997_605191.txt,"Well, here we go with the stream of consciousn...",n,n,y,n,n
3,1997_687252.txt,An open keyboard and buttons to push. The thin...,n,y,n,y,y
4,1997_568848.txt,I can't believe it! It's really happening! M...,y,n,y,y,n
...,...,...,...,...,...,...,...
2463,2004_493.txt,I'm home. wanted to go to bed but remembe...,n,y,n,y,n
2464,2004_494.txt,Stream of consiousnesssskdj. How do you s...,y,y,n,n,y
2465,2004_497.txt,"It is Wednesday, December 8th and a lot has be...",n,n,y,n,n
2466,2004_498.txt,"Man this week has been hellish. Anyways, now i...",n,y,n,n,y


In [ ]:
y = df[[2,3,4,5,6]].replace(('n','y'),(0,1))[1:].set_axis(['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN'], axis=1)

In [ ]:
x = df[[1]][1:].set_axis(['TEXT'], axis=1)

In [ ]:
y


,cEXT,cNEU,cAGR,cCON,cOPN
1,0,1,1,0,1
2,0,0,1,0,0
3,0,1,0,1,1
4,1,0,1,1,0
5,1,0,1,0,1
...,...,...,...,...,...
2463,0,1,0,1,0
2464,1,1,0,0,1
2465,0,0,1,0,0
2466,0,1,0,0,1


In [ ]:
x['TEXT']

1       Well, right now I just woke up from a mid-day ...
2       Well, here we go with the stream of consciousn...
3       An open keyboard and buttons to push. The thin...
4       I can't believe it!  It's really happening!  M...
5       Well, here I go with the good old stream of co...
                              ...                        
2463         I'm home. wanted to go to bed but remembe...
2464         Stream of consiousnesssskdj. How do you s...
2465    It is Wednesday, December 8th and a lot has be...
2466    Man this week has been hellish. Anyways, now i...
2467    I have just gotten off the phone with brady. I...
Name: TEXT, Length: 2467, dtype: object

In [ ]:
text_test = x['TEXT']
  

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1", trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(5, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(text_test))
# print(tf.sigmoid(bert_raw_result))

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 250
steps_per_epoch = 10
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
history = classifier_model.fit(x,np.asarray(y).astype('float32'),epochs=epochs)                        
                              

In [ ]:
model.save('my_model.h5')